In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -P data/tinyshakespeare/

--2024-03-17 15:58:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘data/tinyshakespeare/input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-03-17 15:58:41 (9.39 MB/s) - ‘data/tinyshakespeare/input.txt.4’ saved [1115394/1115394]



In [2]:
with open('data/tinyshakespeare/input.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
dictionary = sorted(list(set(text)))
vocab_size = len(dictionary)
print(''.join(dictionary))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {s:i for i,s in enumerate(dictionary)}
itos = {i:s for i,s in enumerate(dictionary)}
encode = lambda s: [stoi[x] for x in s]
decode = lambda l: ''.join(itos[x] for x in l)
print(encode('Kicior to szczur'))
print(decode(encode('Kicior to szczur')))

[23, 47, 41, 47, 53, 56, 1, 58, 53, 1, 57, 64, 41, 64, 59, 56]
Kicior to szczur


In [7]:
import torch
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9*len(data))
train = data[:n]
test = data[n:]

In [9]:
block_size = 8
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train[:block_size]
y = train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When context is {context} then target is {target}")

When context is tensor([18]) then target is 47
When context is tensor([18, 47]) then target is 56
When context is tensor([18, 47, 56]) then target is 57
When context is tensor([18, 47, 56, 57]) then target is 58
When context is tensor([18, 47, 56, 57, 58]) then target is 1
When context is tensor([18, 47, 56, 57, 58,  1]) then target is 15
When context is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train if split == 'train' else test
    idx = torch.randint(len(data) - block_size, (batch_size,))
    xb = torch.stack([data[i:i+block_size] for i in idx])
    yb = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return xb, yb

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When context is {context.tolist()} then target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When context is [24] then target is 43
When context is [24, 43] then target is 58
When context is [24, 43, 58] then target is 5
When context is [24, 43, 58, 5] then target is 57
When context is [24, 43, 58, 5, 57] then target is 1
When context is [24, 43, 58, 5, 57, 1] then target is 46
When context is [24, 43, 58, 5, 57, 1, 46] then target is 43
When context is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
When context is [44] then target is 53
When context is [44, 53] then target is 56
When context is [44, 53, 56] then target is 1
When context is [44, 53, 56, 1] then target is 58
When context is

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # (C, C)

    def forward(self, x, y=None):  # x: (B, T), y: (B, T)
        logits = self.token_embedding_table(x)  # (B, T, C)
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), y.view(B*T))
        return logits, loss

    def generate(self, x, max_tokens):
        for _ in range(max_tokens):
            logits, loss = self(x)
            logits = logits[:, -1, :]  # (B, 1, C)
            probs = F.softmax(logits, dim=-1)
            yhat = torch.multinomial(probs, num_samples=1)
            x  = torch.cat((x, yhat), dim=1)
        return x


model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(torch.zeros((1,1), dtype=torch.int64), max_tokens=100)[0].tolist()))

torch.Size([4, 8, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [20]:
batch_size = 32
for _ in range(1000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

2.4935240745544434


In [21]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.int64), max_tokens=100)[0].tolist()))


TONThe.
adalor thofe grs ind;
OHAnkil?



WAnerro d s, t aceathad ave! or,

AUKENGren ffo
JUKI y ees


In [44]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, axis=1, keepdim=True)
print(a)
b = torch.randint(0, 10, (3,2)).float()
print(b)
c = a @ b
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [47]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [62]:
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]  # xprev: (T, C)
        xbow[b,t] = torch.mean(xprev, dim=0)
xbow # xbow: (B, T, C)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [78]:
import math

wei = torch.tril(torch.ones(T,T)) # T, T
wei = wei / torch.sum(wei, axis=1, keepdim=True) # T, T
xbow2 = wei @ x  # T, T @ B, T, C
torch.allclose(xbow, xbow2, rtol=math.inf)

True

In [89]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, -math.inf)
wei = F.softmax(wei, dim=-1)  # softmax over channels (last dimension)
xbow3 = wei @ x
torch.allclose(xbow, xbow3, rtol=math.inf)

True

In [106]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)

k = key(x)  # B, T, 16
q = query(x)  # B, T, 16

wei = k @ q.transpose(-2, -1) * head_size**-0.5  # B, T, T

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, -math.inf)
wei = F.softmax(wei, dim=-1)  # softmax over channels (last dimension)

value = nn.Linear(C, head_size, bias=False)
v = value(x)  # B, T, 16

out = wei @ v  # B, T, 16
print(out.shape)
print(out[0])

torch.Size([4, 8, 16])
tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.3156,  0.0704, -0.0706, -0.1604, -0.1344,  0.1558, -0.2001, -0.2886,
         -0.4120,  0.4947,  0.4806, -0.3233, -0.0231, -0.0158,  0.0837,  0.9683],
        [ 0.4029, -0.0241, -0.2422,  0.0145,  0.0144, -0.0129, -0.0916, -0.1296,
         -0.3266,  0.0527,  0.3795, -0.0745, -0.1562, -0.0397, -0.0320,  1.0982],
        [ 0.4779, -0.2057, -0.2656,  0.1018,  0.0853, -0.1675, -0.1532, -0.1084,
         -0.1658,  0.2856,  0.3110, -0.0448,  0.0502,  0.1370,  0.1077,  0.9660],
        [ 0.3579,  0.2417,  0.0711,  0.1706,  0.2915,  0.1982,  0.2582,  0.1424,
         -0.3187, -0.4826, -0.1465, -0.0625, -0.4228,  0.1282,  0.0811,  0.6980],
        [ 0.2370,  0.1631, -0.0279,  0.1223,  0.1766,  0.1643,  0.0528, -0.0590,
         -0.2692, -0.0797,  0.0590, -0.0432, -0.2163,  0.0780,  0.1760,  0.6983],

In [51]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
torch.manual_seed(1337)

vocab_size = 65
batch_size = 16
block_size = 32
n_embed = 256
num_iters = 5000
eval_iters = 200
head_size = 32
n_head = 4
n_blocks = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 3e-4
dropout = 0.1

with open('data/tinyshakespeare/input.txt', 'r', encoding='UTF-8') as f:
    text = f.read()
dictionary = sorted(list(set(text)))
vocab_size = len(dictionary)
stoi = {s:i for i,s in enumerate(dictionary)}
itos = {i:s for i,s in enumerate(dictionary)}
encode = lambda s: [stoi[x] for x in s]
decode = lambda l: ''.join(itos[x] for x in l)
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.int64)

n = int(0.9*len(data))
train = data[:n]
test = data[n:]


def get_batch(split):
    data = train if split == 'train' else test
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class SingleAttentionHead(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # T, T

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  # B, T, head_size
        q = self.query(x)  # B, T, head_size
        
        wei = k @ q.transpose(-2, -1) * head_size**-0.5  # B, T, T
        
        wei = wei.masked_fill(self.tril[:T, :T] == 0, -math.inf)
        wei = F.softmax(wei, dim=-1)  # softmax over channels (last dimension)
        v = self.value(x)  # B, T, head_size
        
        out = wei @ v  # B, T, head_size
        return out


class MultiHeadAttention(nn.Module):

    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SingleAttentionHead(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class TransformerBlock(nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.layer_norm_sa = nn.LayerNorm(n_embed)
        self.feed_forward = FeedForward(n_embed)
        self.layer_norm_ff = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = self.layer_norm_sa(x + self.self_attention(x))
        x = self.layer_norm_ff(x + self.feed_forward(x))
        return x


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)  # (C, n_embed)
        self.position_embedding_table = nn.Embedding(vocab_size, n_embed)  # (C, n_embed)
        self.transformers = nn.Sequential(*[TransformerBlock(n_embed, n_head) for _ in range(n_blocks)])
        self.dropout = nn.Dropout(dropout)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, x, y=None):  # x: (B, T), y: (B, T)
        B, T = x.shape
        tok_emb = self.token_embedding_table(x)  # (B, T, n_embed)
        pos_emb = self.position_embedding_table(x)  # (B, T, n_embed)
        x = tok_emb + pos_emb
        x = self.dropout(x)
        x = self.transformers(x)
        logits = self.lm_head(x)

        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), y.view(B*T))
        return logits, loss

    def generate(self, x, max_tokens):
        for _ in range(max_tokens):
            x_cond = x[:, -block_size:]
            logits, loss = self(x_cond)
            logits = logits[:, -1, :]  # (B, 1, C)
            probs = F.softmax(logits, dim=-1)
            yhat = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, yhat), dim=1)
        return x


model = BigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

6.361921 M parameters


In [ ]:
for i in range(num_iters):
    if i % eval_iters == 0 or i == num_iters - 1:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, eval loss {losses['eval']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

step 0: train loss 4.3201, eval loss 4.3174
step 200: train loss 2.3895, eval loss 2.4144


In [46]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.int64, device=device), max_tokens=1000)[0].tolist()))


EENNIhyOsi f seeleIp s Do, Tes n heerir ha h,iinod sh :aeppee w whroaers h taTf ireseA
egneseeod tUoleIiDaamw shL lt,fiurn toruo nno
ttoHo
 Uao sspeeroni,
l woueidoghh rAnn,ieedr
nat Uodh iuf:deniglaaf;t,r yes pera ivworaeineAVta thete!T;sose s det bosoei Lh enh; 'rt welhuWlers
 wrt
I oen ite m Cuoaunaalyyso ciny fat din ganhuead taeaaol sale,mnhR,amk WYish tr onsdafT eeeal eeer hi pea co lrsn!nl
l

nasuWS whuntdm wae, ghree bweslueose os
n:hgemysrv maiHato' bgnoeTu
 irut
ongit in duhtin h Iremgh? n Iemhew buw
 pairsaHt' OgeSgisTaoeSgiad t IAoerk Wau- teoT tsi  use rd EI, w,heoes n too,h
N hoAsenthdor crn Oenenc EuC elomn'lh,e o homs.rpa
ao,e SeM seon btnnaus, n?at, eyo maeide tiepe sv gris, trlok
fllFmEs!rcclsyiiI akg oits neas oTc mnhiueni, nshi ti f totifieIN

r:' a oro,eotPoms- so
?eAk wcho o?yoel nrrle'
GNfarroereu:s ion?nen tdou
,aoey?deTah intteal gerronerd
SeOhh- ald
 tanb po t atauholni:ee marul I hhhorrrfhd botA
W t ohoighT
ldm ldlR.e mE.maA wii 
rp:cRian g tea srna
fhd fren